In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import pandas as pd
import numpy as np
import re
import tarfile
import pickle as pickle
from tqdm import tqdm
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from sklearn.model_selection import train_test_split

device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()




def load_data(dataset_dir):
    with open('/opt/ml/input/data/label_type.pkl', 'rb') as f:
        label_type = pickle.load(f)
    dataset = pd.read_csv(dataset_dir, delimiter='\t', header=None)
    dataset = preprocessing_dataset(dataset, label_type)
    return dataset

def preprocessing_dataset(dataset, label_type):
    label = []
    for i in dataset[8]:
        if i == 'blind':
            label.append(100)
        else:
            label.append(label_type[i])
    out_dataset = pd.DataFrame({'sentence':dataset[1],'entity_01':dataset[2],'entity_02':dataset[5],'label':label,})
    return out_dataset

dataset_path = r"/opt/ml/input/data/train/train.tsv"

dataset = load_data(dataset_path)

dataset['sentence'] = dataset['entity_01'] + ' [SEP] ' + dataset['entity_02'] + ' [SEP] ' + dataset['sentence']

train, vali = train_test_split(dataset, test_size=0.2, random_state=42)
train[['sentence','label']].to_csv("/opt/ml/input/data/train/train_train.txt", sep='\t', index=False)
vali[['sentence','label']].to_csv("/opt/ml/input/data/train/train_vali.txt", sep='\t', index=False)

dataset_train = nlp.data.TSVDataset("/opt/ml/input/data/train/train_train.txt", field_indices=[0,1], num_discard_samples=1)
dataset_vali = nlp.data.TSVDataset("/opt/ml/input/data/train/train_vali.txt", field_indices=[0,1], num_discard_samples=1)

using cached model
using cached model


In [2]:
bertmodel

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )

In [3]:
dataset_train[0:5]

[['히틀러 [SEP] 독일 [SEP] 무하의 애국심은 히틀러의 독일 정부가 그를의 적으로 간주한 명부에 포함할 만큼 유명했다.', '7'],
 ['촉군 [SEP] 위나라 [SEP] 위나라가 퇴각하면서 일대의 주민과 가축을 몽땅 데려가 버렸기 때문에 무도 음평에 주둔한 촉군은 인력과 식량의 부족에 시달렸고 제갈량은 다시 한중으로 돌아갔다.',
  '0'],
 ['삼성카드 [SEP] 삼성그룹 [SEP] 이는 삼성그룹 회장인 이건희가 아들인 이재용에게 경영권을 인계하여 에버랜드-삼성생명-삼성카드-삼성전자- 에버랜드로 이어지는 순환출자에 의한 그룹 지배를 확보하는데 있어 중요한 역할을 하였다.',
  '3'],
 ["대전방송 [SEP] TJB [SEP] 청원인은 'TJB 대전방송'에서 6년을 근무하며 그중 3년 6개월을 '8뉴스' 앵커로 일하고 퇴사했지만, 퇴직금을 받지 못했다고 주장했다.",
  '6'],
 ['찰스 라이엘 [SEP] 지질학 [SEP] 저명한 교수로는 물리학자 에드워드 빅터 애플턴, 오언 윌런스 리처드슨, 물리학자/수학자 제임스 맥스웰, 생물물리학자 로절린드 프랭클린, 생물학자 줄리언 헉슬리, 지질학자 찰스 라이엘, 외과의사 조지프 리스터, 역사가 아널드 J. 토인비 등이 있다.',
  '2']]

In [4]:
dataset_vali[0]

['정교회 [SEP] 대주교 [SEP] 콥트 정교회 교황 요셉 2세가 최초의 에티오피아인 출신 대주교로 임명한 아부나 바실리오스는 에티오피아 테와히도 정교회의 초대 총대주교가 되었다.',
 '10']

In [5]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [6]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [31]:
max_len = 128
batch_size = 32
warmup_ratio = 0.01
num_epochs = 6
max_grad_norm = 1
log_interval = 50
learning_rate = 5e-5

In [32]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_vali = BERTDataset(dataset_vali, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
vali_dataloader = torch.utils.data.DataLoader(data_vali, batch_size=batch_size, num_workers=5)

In [33]:
len(data_train[0])

4

In [34]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 42,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
#         print(pooler)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
# model = BERTClassifier(bertmodel).to(device)

In [35]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes=42, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = LabelSmoothingLoss()

In [36]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [37]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    best_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(train_dataloader):
        optimizer.zero_grad()
#         print(type(token_ids.long().to(device)))
#         print(type(valid_length))
#         print(type(segment_ids.long().to(device)))
#         print(type(label.long().to(device)))
#         break
#     break
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(vali_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    if test_acc >= best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), "/opt/ml/model/model_state_dict.pt")

epoch 1 batch id 1 loss 3.903625965118408 train acc 0.0
epoch 1 batch id 51 loss 1.7728376388549805 train acc 0.4338235294117647
epoch 1 batch id 101 loss 1.2574117183685303 train acc 0.557240099009901
epoch 1 batch id 151 loss 1.1993439197540283 train acc 0.6072019867549668
epoch 1 batch id 201 loss 0.7840970754623413 train acc 0.6340174129353234
epoch 1 train acc 0.6443055555555556
epoch 1 test acc 0.6677631578947368
epoch 2 batch id 1 loss 1.3913025856018066 train acc 0.6875
epoch 2 batch id 51 loss 0.8508379459381104 train acc 0.7199754901960784
epoch 2 batch id 101 loss 0.8175843954086304 train acc 0.744430693069307
epoch 2 batch id 151 loss 0.8247249126434326 train acc 0.7555877483443708
epoch 2 batch id 201 loss 0.5799598097801208 train acc 0.7644589552238806
epoch 2 train acc 0.7681944444444444
epoch 2 test acc 0.6864035087719298
epoch 3 batch id 1 loss 1.1091325283050537 train acc 0.6875
epoch 3 batch id 51 loss 0.9209443926811218 train acc 0.8088235294117647
epoch 3 batch id 

# Prediction

## test dataset processing

In [ ]:
dataset_path = r"/opt/ml/input/data/test/test.tsv"

dataset = load_data(dataset_path)

dataset['sentence'] = dataset['entity_01'] + ' [SEP] ' + dataset['entity_02'] + ' [SEP] ' + dataset['sentence']

dataset[['sentence','label']].to_csv("/opt/ml/input/data/test/test.txt", sep='\t', index=False)

## test dataset loader

In [26]:
dataset_test = nlp.data.TSVDataset("/opt/ml/input/data/test/test.txt", field_indices=[0,1], num_discard_samples=1)

data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

## infernece

In [27]:
model.load_state_dict(torch.load("/opt/ml/model/model_state_dict.pt"))

model.eval()

Predict = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    _, predict = torch.max(out,1)
    Predict.extend(predict.tolist())

## Reslut

In [30]:
output = pd.DataFrame(Predict, columns=['pred'])
output.to_csv('/opt/ml/result/submission_kobert.csv', index=False)